In [1]:
from __future__ import division
import pandas as pd
import numpy as np
import datetime
import re
import time
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 500)
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import LabelEncoder, MinMaxScaler
import xgboost as xgb
from math import sqrt
from sklearn.metrics import mean_squared_error
from scipy.stats import uniform, randint
from sklearn.model_selection import TimeSeriesSplit, cross_val_score, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import StandardScaler
#import pydot_ng as pydot
from IPython.display import Image
import math
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
#from keras.models import Sequential, Model
#from keras.layers import Input, Dense, Activation, Reshape
#from keras.layers import Concatenate, Dropout
#from keras.layers.embeddings import Embedding
#from keras.utils import plot_model
#from keras.optimizers import SGD
from sklearn.model_selection import KFold, cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_log_error
from sklearn.ensemble import RandomForestRegressor
from lightgbm import LGBMRegressor
from sklearn import base

In [2]:
training_data = pd.read_csv("./data/TrainingSet.csv",index_col=0)
submission_labels = pd.read_csv("./data/SubmissionRows.csv", index_col=0)

training_data.head(2)

,1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],Country Name,Series Code,Series Name
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.769214,Afghanistan,allsi.bi_q1,(%) Benefits held by 1st 20% population - All ...
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.027746,Afghanistan,allsp.bi_q1,(%) Benefits held by 1st 20% population - All ...


In [3]:

# Func to split that dataframe to values [2005,2006,2007] and [country_name,series_code,series_name]
def split_dataframe(data):
    raw_data = data.loc[:,:'2007 [YR2007]']
    #raw_data=data.iloc[:,:-1] 
    description = data.loc[:,'Country Name':]
    #description=data.iloc[:,-1] 
    return raw_data,description


raw_data, description = split_dataframe(training_data)

description.head()

,Country Name,Series Code,Series Name
0,Afghanistan,allsi.bi_q1,(%) Benefits held by 1st 20% population - All ...
1,Afghanistan,allsp.bi_q1,(%) Benefits held by 1st 20% population - All ...
2,Afghanistan,allsa.bi_q1,(%) Benefits held by 1st 20% population - All ...
4,Afghanistan,allsi.gen_pop,(%) Generosity of All Social Insurance
5,Afghanistan,allsp.gen_pop,(%) Generosity of All Social Protection


In [4]:
# Impute the missing values using standard interpolation method

raw_data = raw_data.interpolate(limit_direction='both',axis=1)

raw_data.head()

,1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007]
0,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214
1,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746,7.027746
2,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887,8.244887
4,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105,12.933105
5,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814,18.996814


In [5]:
training_data=raw_data.join(description)

training_data.head(1) 


,1972 [YR1972],1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],1979 [YR1979],1980 [YR1980],1981 [YR1981],1982 [YR1982],1983 [YR1983],1984 [YR1984],1985 [YR1985],1986 [YR1986],1987 [YR1987],1988 [YR1988],1989 [YR1989],1990 [YR1990],1991 [YR1991],1992 [YR1992],1993 [YR1993],1994 [YR1994],1995 [YR1995],1996 [YR1996],1997 [YR1997],1998 [YR1998],1999 [YR1999],2000 [YR2000],2001 [YR2001],2002 [YR2002],2003 [YR2003],2004 [YR2004],2005 [YR2005],2006 [YR2006],2007 [YR2007],Country Name,Series Code,Series Name
0,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,3.769214,Afghanistan,allsi.bi_q1,(%) Benefits held by 1st 20% population - All ...


In [ ]:
#Let's try to interpolate missing data  and ensure there are no nulls

#training_data_pivotted = training_data.melt(id_vars=['Series Code', 'Country Name'], var_name='Year', value_name='indicator',ignore_index = False, value_vars=['2005 [YR2005]','2006 [YR2006]','2007 [YR2007]'])
#df=training_data_pivotted[training_data_pivotted.indicator.isnull()]
#df.head()

In [6]:
training_data_pivotted = training_data.melt(id_vars=['Series Code', 'Country Name'], var_name='Year', value_name='indicator',ignore_index = False, \
                        value_vars=['1972 [YR1972]','1973 [YR1973]','1974 [YR1974]','1975 [YR1975]',\
                                    '1976 [YR1976]','1977 [YR1977]','1978 [YR1978]','1979 [YR1979]', \
                                    '1980 [YR1980]',  '1981 [YR1981]', '1982 [YR1982]', '1983 [YR1983]', \
                                    '1984 [YR1984]','1985 [YR1985]','1986 [YR1986]','1987 [YR1987]', \
                                    '1988 [YR1988]','1989 [YR1989]','1990 [YR1990]','1991 [YR1991]', \
                                    '1992 [YR1992]','1993 [YR1993]','1994 [YR1994]','1995 [YR1995]', \
                                    '1996 [YR1996]', '1997 [YR1997]','1998 [YR1998]','1999 [YR1999]', \
                                    '2000 [YR2000]','2001 [YR2001]','2002 [YR2002]','2003 [YR2003]' , \
                                    '2004 [YR2004]','2005 [YR2005]','2006 [YR2006]','2007 [YR2007]'])


'''
training_data_pivotted = training_data.melt(id_vars=['Series Code', 'Country Name'], var_name='Year', value_name='indicator',ignore_index = False, value_vars=['2005 [YR2005]','2006 [YR2006]','2007 [YR2007]'])
'''

training_data_pivotted['Year'] = training_data_pivotted['Year'].str.extract('(\d+)', expand=False) #.4stype(i4t),'2005 [YR2005]','2006 [YR2006]'


# Let try to interpolate the missing values 


#df = df.sort_values(['Week', 'Product_Code'])

pd.set_option('display.max_colwidth', None)

training_data_pivotted['Year'] = pd.to_datetime(training_data_pivotted['Year'], format='%Y')

#Ignore data is null 

#df=df[df.indicator.notnull()]

#df.head()

#training_data_pivotted.to_csv("./data/training_data_pivotted.csv")

In [7]:

#imputed_results_raw=pd.read_csv("../data/imputed_res.csv",index_col='int64_field_0')

imputed_results_raw=training_data_pivotted

# Only fetch the row required for prediction as presence of other indicators causes data skew problems 

imputed_results=imputed_results_raw.join(submission_labels,how='inner')

imputed_results=imputed_results.rename({'Series_Code':'Series Code','Country_Name': 'Country Name'},axis=1)

imputed_results=imputed_results.drop(['2008 [YR2008]','2012 [YR2012]'], axis=1)

#imputed_results.groupby('Year').nunique()

#make sure the index is preserved. May not be required
#imputed_results[imputed_results.index == 559]

In [8]:
print("min:: " + str(imputed_results["indicator"].min()) + "\nmax:: " + str(imputed_results["indicator"].max()))

min:: 0.0
max:: 1.0


In [9]:
#Ensure null values are imputed 
df=imputed_results[imputed_results.indicator.isnull()]
df.head()

,Series Code,Country Name,Year,indicator


In [10]:
#Select the number of years you want to compute it for 
imputed_results=imputed_results[pd.to_datetime(imputed_results['Year'], format='%Y-%m-%d') >= '1990-01-01']
#imputed_results=imputed_results[imputed_results['Country Name'] == 'Afghanistan']
imputed_results.head()

,Series Code,Country Name,Year,indicator
559,7.8,Afghanistan,1990-01-01,0.048
559,7.8,Afghanistan,1991-01-01,0.048
559,7.8,Afghanistan,1992-01-01,0.049
559,7.8,Afghanistan,1993-01-01,0.049
559,7.8,Afghanistan,1994-01-01,0.049


In [11]:
#Data Enrichment 
# Append the continent and sub region 
countryContinent= pd.read_csv("./data/countryContinentutf8.csv")

countryContinent=countryContinent.fillna(0)

countryContinent=countryContinent.rename({'country':'Country Name'},axis=1)

countryContinent[countryContinent.region_code.isnull()]

#countryContinent.groupby("Country Name").nunique()


,Country Name,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code


In [12]:
#imputed_results_joined=imputed_results.join(countryContinent,on=['Country Name'],how='left')
imputed_results= pd.merge(left=imputed_results, right=countryContinent, how='left', left_on='Country Name', right_on='Country Name')

imputed_results.count()

Series Code        13266
Country Name       13266
Year               13266
indicator          13266
code_2             13194
code_3             13194
country_code       13194
iso_3166_2         13194
continent          13194
sub_region         13194
region_code        13194
sub_region_code    13194
dtype: int64

In [13]:
#Ensure null values are imputed 
imputed_results=imputed_results.fillna(0)

df=imputed_results[imputed_results.region_code.isnull()]
df.groupby("Country Name").nunique()

,Series Code,Year,indicator,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code
Country Name,,,,,,,,,,,


In [14]:
#Apply encoding of categorical values 
# Label Encoding 

encode_columns = ['Country Name','Series Code']
encode_df = imputed_results[encode_columns]
encode_df = encode_df.astype('str')
encode_df = encode_df.apply(LabelEncoder().fit_transform)
score_encode_drop = imputed_results.drop(encode_columns, axis = 1)
#score_encode_drop=df

score_encode = pd.concat([score_encode_drop, encode_df], axis = 1)
score_encode['Year']=pd.to_datetime(imputed_results['Year'], format='%Y-%m-%d').dt.year
score_target=score_encode
#score_target.groupby('Country Name').nunique()
score_target.head()

,Year,indicator,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code,Country Name,Series Code
0,1990,0.048,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0,5
1,1991,0.048,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0,5
2,1992,0.049,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0,5
3,1993,0.049,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0,5
4,1994,0.049,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0,5


In [15]:
#Split Train and Test Data
split_date = 2006
df_train = score_target.loc[score_target['Year'] <= split_date].copy()
df_test = score_target.loc[score_target['Year'] > split_date].copy()

def create_features(df, label=None):
    #df['Year'] =  pd.to_datetime(df['Year'], format='%Y').dt.year
    
    #Original config without the enrichment 
    #X = df[['Country Name','Series Code','Year']]  
    
    #With enrichment changing the features
    X = df[['country_code','Series Code','region_code','sub_region_code', 'Year']] 
    
    if label:
        y = df[label]
        return X, y
    return X
X_train, y_train = create_features(df_train, label='indicator')
X_test, y_test = create_features(df_test, label='indicator')

#X_train.count()
X_train.head()
#X_train.groupby('Year').nunique()
#print(create_features(df_train, label='indicator'))

,country_code,Series Code,region_code,sub_region_code,Year
0,4.0,5,142.0,34.0,1990
1,4.0,5,142.0,34.0,1991
2,4.0,5,142.0,34.0,1992
3,4.0,5,142.0,34.0,1993
4,4.0,5,142.0,34.0,1994


In [40]:
#Test with simple Regression Model 
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

y_pred = regressor.predict(X_test)

rms = sqrt(mean_squared_error(y_test, y_pred))
print ('RMSE:', rms)

RMSE: 0.38943286100512803


In [53]:
# Xboost Model

params = {'colsample_bytree': uniform(0.7, 0.3),
          'gamma': uniform(0, 0.7),
          'learning_rate': uniform(0.003, 0.5), # default 0.1 
          'max_depth': randint(60, 100), # default 3
          'n_estimators': randint(9999, 10000), # default 100
          'subsample': uniform(0.6, 0.4)}
xgb_model = xgb.XGBRegressor(objective="reg:squarederror",random_state=300)
time_split = TimeSeriesSplit(n_splits = 2)
xgb_search = RandomizedSearchCV(xgb_model,param_distributions=params, random_state=300, n_iter=4, cv=time_split, verbose=1, n_jobs=1, return_train_score=True)
xgb_search.fit(X_train,y_train)

y_pred = xgb_search.predict(X_test)
#y_pred = model.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))

print ('RMSE:', rms)

Fitting 2 folds for each of 4 candidates, totalling 8 fits
RMSE: 0.06903589180984786


In [20]:
#Random forest Model

#n_estimators=[500, 800,1500,2500,5000]
#max_features=['auto','sqrt','log2']
#max_depth= [10,20,30,40,50]
#max_depth.append(None)
#min_samples_split=[2,5,10,15,20]
#min_samples_leaf=[1,2,5,10,15]

n_estimators=[1000]
max_features=['auto']
max_depth= [10]
max_depth.append(None)
min_samples_split=[2]
min_samples_leaf=[1]



grid_param = {'n_estimators': n_estimators, \
             'max_features': max_features, \
             'max_depth': max_depth, \
             'min_samples_split': min_samples_split, \
             'min_samples_leaf': min_samples_leaf}

'''
model = RandomForestRegressor(n_estimators=1000,random_state=100,max_features='sqrt')
model.fit(X_train, y_train)

'''
model = RandomForestRegressor(random_state=1)
model_random=RandomizedSearchCV(estimator=model,param_distributions=grid_param, n_iter=500, cv=5, verbose=2, random_state=42, n_jobs=-1)

model_random.fit(X_train,y_train)
print(model_random.best_params_)


/opt/conda/lib/python3.7/site-packages/sklearn/model_selection/_search.py:289: UserWarning: The total space of parameters 2 is smaller than n_iter=500. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


Fitting 5 folds for each of 2 candidates, totalling 10 fits
{'n_estimators': 1000, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_features': 'auto', 'max_depth': 10}


In [39]:
model = RandomForestRegressor(n_estimators=1000,random_state=1,max_features='auto',max_depth=100, min_samples_split=2,min_samples_leaf=1)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
rms = sqrt(mean_squared_error(y_test, y_pred))

print ('RMSE:', rms)

RMSE: 0.028536066946563167


In [98]:
#2005 Split  + 1972 data 
# RMSE: 0.04403392579660952

#2006 Split + 1972 data
# RSME :  0.03701239963313429

#2006 Split + 2000 data 
#0.0367124955614689

#200 estimators, random 100
#RMSE: 0.038374986721595734

#RMSE: 0.03692818441424601


#With Enriched Data 
# 0.06533870558142173

,Country Name,Series Code,indicator
Year,,,
2008,206,7,1


In [54]:
submission_results=submission_labels.join(training_data)

submission_results=submission_results[['Country Name','Series Code']]

submission_results['Year']=2008

submission_results['indicator']=0

#submission_results.head() 

#submission_results.groupby('Year').nunique()

# Label Encoding 

encode_columns = ['Series Code']
pred_df = submission_results[encode_columns]
score_predict_drop = submission_results.drop(encode_columns, axis = 1)
pred_df = pred_df.astype('str')
pred_df = pred_df.apply(LabelEncoder().fit_transform)


score_predict_2008 = pd.concat([score_predict_drop, pred_df], axis = 1)

score_predict_2008['Year']=pd.to_datetime(score_predict_2008['Year'],format='%Y').dt.year

#res_pred_01 = xgb_search.predict(score_predict_2008[['Country Name','Series Code','Year']])


score_predict_2008=pd.merge(left=score_predict_2008, right=countryContinent, how='left', left_on='Country Name', right_on='Country Name').set_axis(score_predict_2008.index)


score_predict_2008=score_predict_2008.fillna(0)


#Replace the model name here for different models 

#Random forest
#res_pred_01 = model.predict(score_predict_2008[['country_code','Series Code','region_code','sub_region_code', 'Year']])

#Xboost
res_pred_01 = xgb_search.predict(score_predict_2008[['country_code','Series Code','region_code','sub_region_code', 'Year']])

score_predict_2008['predictions'] = res_pred_01

score_predict_2008.head()


,Country Name,Year,indicator,Series Code,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code,predictions
559,Afghanistan,2008,0,5,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.310938
618,Afghanistan,2008,0,6,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.074294
753,Afghanistan,2008,0,1,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.074222
1030,Afghanistan,2008,0,3,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.009687
1896,Albania,2008,0,5,AL,ALB,8.0,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0,0.940391


In [55]:
submission_results=submission_labels.join(training_data)

submission_results=submission_results[['Country Name','Series Code']]

submission_results['Year']=2012

submission_results['indicator']=0

#submission_results.head() 

#submission_results.groupby('Year').nunique()

# Label Encoding 

encode_columns = ['Series Code']
pred_df = submission_results[encode_columns]
score_predict_drop = submission_results.drop(encode_columns, axis = 1)
pred_df = pred_df.astype('str')
pred_df = pred_df.apply(LabelEncoder().fit_transform)


score_predict_2012 = pd.concat([score_predict_drop, pred_df], axis = 1)

score_predict_2012['Year']=pd.to_datetime(score_predict_2012['Year'],format='%Y').dt.year

#res_pred_01 = xgb_search.predict(score_predict_2008[['Country Name','Series Code','Year']])


score_predict_2012=pd.merge(left=score_predict_2012, right=countryContinent, how='left', left_on='Country Name', right_on='Country Name').set_axis(score_predict_2012.index)


score_predict_2012=score_predict_2012.fillna(0)

#Random forest
#res_pred_01 = model.predict(score_predict_2012[['country_code','Series Code','region_code','sub_region_code', 'Year']])


#xgboost
res_pred_01 = xgb_search.predict(score_predict_2012[['country_code','Series Code','region_code','sub_region_code', 'Year']])


score_predict_2012['predictions'] = res_pred_01

score_predict_2012.head()


,Country Name,Year,indicator,Series Code,code_2,code_3,country_code,iso_3166_2,continent,sub_region,region_code,sub_region_code,predictions
559,Afghanistan,2012,0,5,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.310938
618,Afghanistan,2012,0,6,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.074294
753,Afghanistan,2012,0,1,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.074222
1030,Afghanistan,2012,0,3,AF,AFG,4.0,ISO 3166-2:AF,Asia,Southern Asia,142.0,34.0,0.009687
1896,Albania,2012,0,5,AL,ALB,8.0,ISO 3166-2:AL,Europe,Southern Europe,150.0,39.0,0.940391


In [57]:
predictions_from_model=score_predict_2012.join(score_predict_2008,how='inner',lsuffix=2012, rsuffix=2008)


predictions_from_model=predictions_from_model.drop(columns=['Year2012','indicator2012','Country Name2012','Year2008','indicator2008','Country Name2008','Series Code2012','Series Code2008'])


predictions_from_model=predictions_from_model.rename(columns={"predictions2008": "2008 [YR2008]", "predictions2012": "2012 [YR2012]"})

predictions_from_model=predictions_from_model[['2008 [YR2008]','2012 [YR2012]']]

predictions_from_model.to_csv("./data/result_xboost_0120_05.csv")
